<a href="https://colab.research.google.com/github/chetan01101993/HousePricePrediction/blob/master/Profile_Summarizer_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install transformers
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 1.8MB 8.0MB/s 
     |████████████████████████████████| 890kB 27.4MB/s 
     |████████████████████████████████| 3.2MB 47.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ca349f0af7a3ebe3c1f185347b2b1b2fa93680c67cc106c3ce2a8960c1bd1272
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180945 sha256=40320c50cbdaed99830f442592e918e74a5cc84463d08a95885f286cf2ad1100
  Stored in directory: /tmp/pip-ephem-wheel-cache-6f6c92je/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
!pip install mysql-connector-python

     |████████████████████████████████| 18.1MB 56.9MB/s 


In [ ]:
from transformers import BartTokenizer, BartConfig, BartForConditionalGeneration
import pprint
import en_core_web_lg 
from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer
import re 
import pandas as pd
from google.colab import files
import io
import numpy as np
import mysql.connector as mysql   
import time

In [ ]:
from configparser import ConfigParser
config = ConfigParser()

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
nlp = en_core_web_lg.load()

In [ ]:
uploaded = files.upload()

Saving indian_companies_profile_latest.csv to indian_companies_profile_latest.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['indian_companies_profile_latest.csv']))
df = df.replace(np.nan,'',regex=True)

In [ ]:
df.head()

,company_name,entity_id,manual,bloomberg,marketscreener,yahoofinance
0,AARTI Industries Ltd.,05WLR7-E,,,Aarti Industries Limited is a manufacturer of ...,Aarti Industries Limited manufactures and sell...
1,ABB India Ltd.,05J1G1-E,,,"ABB India Limited specializes in the design, m...",ABB India Limited offers industrial technology...
2,ACC Ltd.,05HZC9-E,,,ACC is leader in India of cement production. N...,ACC Limited manufactures and sells cement and ...
3,AGC Networks Ltd.,09SKF5-E,,,AGC Networks Limited is a Global Solutions Int...,AGC Networks Limited provides information and ...
4,APL Apollo Tubes Ltd.,06SR7L-E,,,APL Apollo Tubes Limited is a steel tube manuf...,APL Apollo Tubes Limited manufactures and sell...


In [ ]:
config.read('config.ini')

['config.ini']

In [ ]:
def clean_text(text):
  text = text.replace('plc','PLC').replace('Plc','PLC').replace('Pte.','Private').replace('Pte ','Private ').replace('Ltd ','Limited ').replace('Inc ','Incorporation ').replace('PT.','PT').replace('Yes Bank','YES BANK').replace('Co ','Corporation ').replace('P.T.','PT').replace('S.A.','SA')
  return text
    
def create_NLP_object(text):
  return nlp(text)

'''
def get_first_sentence(doc):
  for token in doc.sents:
      return token.text
'''
def get_first_sentence(text):
  pk = PunktSentenceTokenizer(text)
  sentences = pk.tokenize(text)
  if sentences!=[]:
    return sentences[0]

def remove_spaces(text):
  text = text.lstrip()
  text = text.rstrip()
  text = re.sub(' +', ' ', text)
  return text


In [ ]:
pattern_pos=r'((DT|PRP){1}.*VBZ.*IN.*NNS?.*(NNP?){1,})'
def get_segments_sentence(sentence_list):
    segment_sentence = []
    for sent in sentence_list:
        doc=create_NLP_object(sent)
        pos_list=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                pos_list.append(token.tag_)
        pos_str=' '.join(pos_list)
        if re.search(pattern_pos,pos_str) and re.search(r'(segment|division)', sent):
        #if re.search(pattern_pos,pos_str):
            #print(sent)
            segment_sentence.append(sent)
    return segment_sentence

In [ ]:
def get_sales_sentences(text):
    number_sentences=[]
    for s in text.split('\n'):
        if re.findall('(\(.*\d+(\.\d+)?%?.*\))',s):
            number_sentences.append(s) 
    if re.search('(\..*breaks? down.*:)',text):
        if number_sentences:
            if 'break down' not in number_sentences[0] or 'breaks down' not in number_sentences[0]:
                #print(re.search('(\..*breaks? down.*:)',text).group(0).split('.')[-1])
                number_sentences[0]=re.search('(\..*breaks? down.*:)',text).group(0).split('.')[-1]+ " " + number_sentences[0]
                number_sentences[0]=number_sentences[0].lstrip('.')
    for sent in number_sentences:
        if re.search('(breaks? down)',sent) or 'geographically' in sent:
            return(number_sentences)
        else:
            continue
    return []    


In [ ]:
origin_re = 'founded|originated|incorporated|headquartered'
verb_list = ['services', 'service', 'segments', 'segment', 'provides', 'offers', 'engages','operates','markets','distributes','sells','proposes', 'specializes', 'serves', 'produces','production','engaged', 'derives', 'business', 'develops','products include','creates','involved','delivers', 'activities include', 'merchandise includes', 'offerings', 'trades','comprise','portfolio includes','inspects','manufactures','explores','solutions include']
subsidiary_list = ['subsidiary','parent','subsidiaries','names','name', 'brand','brands', 'portfolio companies','collaboration', 'own-label', 'shareholder','shareholders','business units', 'business unit','strategic agreement','member']
no_employees = 'Number of employees : \d{0,3}\s*\d{1,3} people.'  

In [ ]:
def tag_sentences(text_manual, text_yahoo, text_bloomberg, text_marketscreener):
  text_marketscreener = re.sub(no_employees, '', text_marketscreener)
  text_manual = clean_text(text_manual)
  text_yahoo = clean_text(text_yahoo)
  text_bloomberg = clean_text(text_bloomberg)
  text_marketscreener = clean_text(text_marketscreener)

  doc_manual = create_NLP_object(text_manual)
  doc_yahoo = create_NLP_object(text_yahoo)
  doc_bloomberg = create_NLP_object(text_bloomberg)
  doc_marketscreener = create_NLP_object(text_marketscreener)

  #first sentence
  manual_first_sentence = get_first_sentence(text_manual)
  yahoo_first_sentence = get_first_sentence(text_yahoo)
  bloomberg_first_sentence = get_first_sentence(text_bloomberg)
  marketscreener_first_sentence = get_first_sentence(text_marketscreener)

  if marketscreener_first_sentence:
    if re.search('\n',marketscreener_first_sentence):
      marketscreener_first_sentence = ''

  if manual_first_sentence: manual_first_sentence = re.sub('(\\n)+',' ', manual_first_sentence)
  if yahoo_first_sentence: yahoo_first_sentence = re.sub('(\\n)+',' ', yahoo_first_sentence)
  if bloomberg_first_sentence: bloomberg_first_sentence = re.sub('(\\n)+',' ', bloomberg_first_sentence)
  if marketscreener_first_sentence: marketscreener_first_sentence = re.sub('(\\n)+',' ', marketscreener_first_sentence)

  sentence_tag = {}
  if manual_first_sentence: sentence_tag[manual_first_sentence] = 'first sentence'
  if yahoo_first_sentence: sentence_tag[yahoo_first_sentence] = 'first sentence'
  if bloomberg_first_sentence: sentence_tag[bloomberg_first_sentence] = 'first sentence'
  if marketscreener_first_sentence: sentence_tag[marketscreener_first_sentence] = 'first sentence'

  print('---------------------')
  print('First sentences')
  for sentence in sentence_tag.keys():
      print(sentence)

  ## sales 
  print('---------------------')
  print('Sales')
  sales_sentence = get_sales_sentences(text_marketscreener)
  text_marketscreener = re.sub('(\\n)+',' ', text_marketscreener)
  for sent in sales_sentence:
      text_marketscreener = text_marketscreener.replace(sent, '')

  text_marketscreener = re.sub(' +', ' ', text_marketscreener)
  sales_sentence = ' '.join(sales_sentence)
  sales_sentence = sales_sentence.lstrip()
  if sales_sentence not in sentence_tag and sales_sentence:
      sentence_tag[sales_sentence] = 'sales'
      print(sales_sentence)

  text = text_manual + " " + text_yahoo + " " + text_bloomberg + " " + text_marketscreener
  text = text.replace('â??',"'")  

  text = remove_spaces(text)
  for sentence,tag in sentence_tag.items():
    if tag in 'first sentence':
      text = text.replace(sentence,' ')
  text = remove_spaces(text)
  
  doc = create_NLP_object(text)
  nltk_s = PunktSentenceTokenizer(text)
  nltk_sentences = nltk_s.tokenize(text)
  
  print('---------------------')
  print('Services')
  #identifying segments of company
  segment_sentences = get_segments_sentence(nltk_sentences)

  #high probability of services
  for sentence in nltk_sentences:
      print('---------------------')
      sentence_verbs_list = [verb for verb in verb_list if verb in sentence]
      if sentence in segment_sentences and sentence not in sentence_tag:
          sentence_tag[sentence] = 'segments.services'
          print(sentence)
      if len(sentence.split(", ")) > 2 and sentence_verbs_list and sentence not in sentence_tag:
          sentence_tag[sentence] = 'strictly services'
          print(sentence)

  #shareholders/subsidiary/related companies
  print('---------------------')
  print('Relations')        
  organizations = list(set([word.text for word in doc.ents if word.label_ == 'ORG']))
  print(organizations)
  for sentence in nltk_sentences:
      sentence_organizations = [organization for organization in organizations if organization in sentence]
      sentence_subsidiary_list = [sub for sub in subsidiary_list if sub in sentence]
      if sentence_subsidiary_list and sentence not in sentence_tag and sentence_organizations and 'changed its name' not in sentence :
          print(sentence)
          sentence_tag[sentence] = 'client.shareholders.subsidiary'

  #monetary & sales
  print('---------------------')
  print('Monetary & Sales')        

  monetary_list = list(set([word.text for word in doc.ents if word.label_ == 'MONEY']))
  for sentence in nltk_sentences:        
      sentence_monetary_list = [money for money in monetary_list if money in sentence]
      if sentence not in sentence_tag and re.search('\d+', sentence) and (sentence_monetary_list or re.search('% of net sales|% of net income|% of income|% of revenue|INR|USD|EUR', sentence)):
          sentence_tag[sentence] = 'money.sales'
          print(sentence)

  #Other numbers but not origin related
  print('---------------------')
  print('Other numbers')        

  for sentence in nltk_sentences:        
      if sentence not in sentence_tag and re.search('\d+', sentence) and re.search(origin_re, sentence) is None:
          sentence_tag[sentence] = 'other.numbers'
          print(sentence)
  return text, sentence_tag

In [ ]:
def connect_db(host,user,password,db):
    connection = None
    try:
        connection = mysql.connect(
            host= host,
            user= user,
            passwd= password,
            database = db
            )
        
        print("connection success")
    except Exception as e:
        print(f'Error {e} occured')    
    return connection

def execute_query(connection,query):
    cursor = connection.cursor()
    result = None
    try:
        print(cursor.execute(query))  
        connection.commit()
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(f'Error {e} occured')

def insert_data(table_name, column_names, values_list, connection):
    print("here")
    query="INSERT INTO {} ({}) VALUES (".format(table_name,','.join(column_names))
    for ele in values_list[0:-1]:
        if type(ele)==str:
            query=query+ "'"+ele+"',"
        else:
            query=query+str(ele)+','
    if type(values_list[-1])==str:
        query=query+"'"+values_list[-1]+"')"
    else:
        query=query+str(values_list[-1])+")"
    execute_query(connection,query)

def insert_data(table_name, column_names, values_list, connection):
    print("here")
    query="INSERT INTO {} ({}) VALUES (".format(table_name,','.join(column_names))
    for ele in values_list[0:-1]:
        if type(ele)==str:
            query=query+ "'"+ele+"',"
        else:
            query=query+str(ele)+','
    if type(values_list[-1])==str:
        query=query+"'"+values_list[-1]+"')"
    else:
        query=query+str(values_list[-1])+")"
    execute_query(connection,query)


In [ ]:
connection = connect_db(
            host = config['database']['host'],
            user = config['database']['user'],
            password = config['database']['password'],
            db = config['database']['db']
            )

connection success


In [ ]:
eid = []
eid = ['05WLR7-E'
,'05J1G1-E'
,'05HZC9-E'
,'09SKF5-E'
,'06SR7L-E'
,'07CL9B-E'
,'05J3ZV-E'
,'05W5DP-E'
,'06WSWZ-E'
,'06YW41-E'
,'0FP1QM-E'
,'0D5DY6-E'
,'006GRF-E'
,'09VWKH-E'
,'089RP4-E'
,'0605GV-E'
,'06HHHW-E'
,'05T3L4-E'
,'05JFZ7-E'
,'0BXG4T-E'
,'05HZ5F-E']

In [ ]:
len(eid)

21

In [ ]:
table_name = config['DEFAULT']['table_name']
column_names = config['DEFAULT']['column_names'].split(',')
country = 'India'

In [ ]:
for index, profiles in df.iterrows():
  entity_id = profiles['entity_id']
  entity_name = profiles['company_name']
  if entity_id in eid:
    print('skip ' + entity_id)
    continue   

  text_bloomberg = profiles['bloomberg']
  text_marketscreener = profiles['marketscreener']
  text_yahoo = profiles['yahoofinance']
  text_manual = profiles['manual']
  print(entity_id)
  print(text_yahoo)
  print(text_manual)
  print(text_bloomberg)
  print(text_marketscreener)
  with open('text.txt','a') as f:
    f.write('\n############################################################################################\n')
    f.write("\n" + entity_id + " " + profiles['company_name'] + "\n")
    f.write('\n' + text_manual)
    f.write('\n' + text_yahoo)
    f.write("\n" + text_bloomberg)
    f.write("\n" + text_marketscreener)
  text, sentence_tag = tag_sentences(text_manual, text_yahoo, text_bloomberg, text_marketscreener)
    
  first_sentences = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in 'first sentence')
  sentences = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in ['segments.services','strictly services'])
  sales = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in ['sales','money.sales'])
  relations = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in 'client.shareholders.subsidiary')
  relations = relations.replace('"','\"').replace("'","''")
  other_numbers = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in 'other.numbers')

  #first sentence summary - minimum length: either 15% or 25
  min_len = round(0.15 * len(first_sentences))
  print("15% is " + str(min_len))
  if min_len > 25:
    min_len = 25
  print(min_len)
  if min_len > 5:
    fs_inputs = tokenizer([first_sentences], return_tensors='pt')
    fs_summary_ids = model.generate(fs_inputs['input_ids'], num_beams=4, no_repeat_ngram_size=2, min_length=min_len, early_stopping=True) 
    fs_summary = tokenizer.decode(fs_summary_ids[0], skip_special_tokens=True)
    pprint.pprint(fs_summary)
    doc_fs = create_NLP_object(fs_summary)
    fs_summary_sentences = [token.text for token in doc_fs.sents]
    for s_idx, sentence in enumerate(fs_summary_sentences):
      if re.search('For more information|Back to|back to |For confidential.? support|founded in \d{4}|WARNING:|Warning:|This article was originally', sentence):
        print(sentence)
        fs_summary = fs_summary.replace(sentence,'')
        fs_summary_sentences.pop(s_idx)
    
    if re.search('The Company is based in|The company is based in|It is based in', fs_summary_sentences[-1]):
      print(fs_summary_sentences[-1])
      fs_summary = fs_summary.replace(fs_summary_sentences[-1],'')

    if fs_summary[-1]!='.' and fs_summary[-1]!=' ':
      fs_summary = fs_summary + "."
  else:
    fs_summary = ''

  #segments & services length summary - minimum length: either 15% or 100
  summary_len = ''
  total_summary = ''
  if sentences:
    inputs = tokenizer([sentences], return_tensors='pt', max_length= len(sentences), truncation=True)
    min_len = round(0.15 * len(sentences))
    print("\n15% is " + str(min_len))
    if min_len > 100:
      min_len = 100
    summary_ids_len = model.generate(inputs['input_ids'], num_beams=4, no_repeat_ngram_size=2, min_length = min_len, max_length=600, early_stopping=True) 
    summary_len = tokenizer.decode(summary_ids_len[0], skip_special_tokens=True)
    print('Services summary:')
    pprint.pprint(summary_len)
    if summary_len[-1]!='.' and summary_len[-1]!= ' ':
      summary_len = summary_len + "."

    doc_summary = create_NLP_object(summary_len)
    summary_sentences = [token.text for token in doc_summary.sents]
    for s_idx, sentence in enumerate(summary_sentences):
      if re.search('For more information|for more|Back to|back to |For confidential.? support|WARNING:|Warning:|This article was originally', sentence):
        print(sentence)
        summary_len = summary_len.replace(sentence,'')
        summary_sentences.pop(s_idx)
      if re.search(r'In \d{4}, (.*)\d|recorded a net profit|recorded a net loss|total assets stand|reported a net profit|reported a net loss|share price|In the U.S. contact the National Suicide Prevention Lifeline', sentence):
        print(sentence)
        summary_len = summary_len.replace(sentence,'')
        summary_sentences.pop(s_idx)
    
    if re.search('The Company is based in|The company is based in|It is based in|In the U.S. contact the National Suicide Prevention Lifeline', summary_sentences[-1]):
      print(summary_sentences[-1])
      summary_len = summary_len.replace(summary_sentences[-1],'')
    
    total_summary = fs_summary + " " + summary_len
    total_summary = total_summary.replace("\xa0"," ")
    total_summary = remove_spaces(total_summary)
    total_summary = total_summary.replace('"','\"').replace("'","''")
    print('\n')
    pprint.pprint(total_summary)

  else:
    print('\nNo services found')
    text = first_sentences + " " + text
    text = remove_spaces(text)
    inputs = tokenizer([text], return_tensors='pt', max_length= len(text), truncation=True)
    min_len = round(0.20 * len(text))
    print("\n20% is " + str(min_len))
    if min_len > 100:
      min_len = 100
    summary_ids_len = model.generate(inputs['input_ids'], num_beams=4, no_repeat_ngram_size=2, min_length = min_len, max_length=300, early_stopping=True) 
    total_summary = tokenizer.decode(summary_ids_len[0], skip_special_tokens=True)
    print('Total summary:')
    pprint.pprint(total_summary)
    if total_summary[-1]!='.':
      total_summary = total_summary + "."
    doc_summary = create_NLP_object(total_summary)
    summary_sentences = [token.text for token in doc_summary.sents]
    for s_idx, sentence in enumerate(summary_sentences):
      if re.search('For more information|for more|Back to|back to |For confidential.? support|WARNING:|Warning:|This article was originally', sentence):
        print(sentence)
        total_summary = total_summary.replace(sentence,'')
        summary_sentences.pop(s_idx)
      if re.search(r'In \d{4}, (.*)\d|recorded a net profit|recorded a net loss|total assets stand|reported a net profit|reported a net loss|share price|In the U.S. contact the National Suicide Prevention Lifeline', sentence):
        print(sentence)
        total_summary = total_summary.replace(sentence,'')  
        summary_sentences.pop(s_idx)

    if re.search('The Company is based in|The company is based in|It is based in|In the U.S. contact the National Suicide Prevention Lifeline', summary_sentences[-1]):
      print(summary_sentences[-1])
      total_summary = total_summary.replace(summary_sentences[-1],'')
    total_summary = total_summary.replace('"','\"').replace("'","''")
 
  with open('text.txt','a') as f:
    f.write('\n\ntotal summary:\n')
    f.write(total_summary)
    f.write('\n\nsubsidiaries:\n')
    f.write(relations)
  values_list = [time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()),time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()),entity_id, entity_name,country,total_summary,relations]
  insert_data(table_name, column_names,values_list,connection)
 
  eid.append(entity_id)
  


skip 05WLR7-E


In [ ]:
files.download('text.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(len(eid))

30


In [ ]:

import os
os.remove('text.txt')


In [ ]:
for index, profiles in df.iterrows():
  entity_id = profiles['entity_id']
  entity_name = profiles['company_name']
  if entity_id == '05J5LN-E':
    print('skip ' + entity_id)
    break


skip 05J5LN-E


In [ ]:
text_bloomberg = profiles['bloomberg']
text_marketscreener = profiles['marketscreener']
text_yahoo = profiles['yahoofinance']
text_manual = profiles['manual']
print(entity_id)
print(text_yahoo)
print(text_manual)
print(text_bloomberg)
print(text_marketscreener)

05J5LN-E
Interroll Holding AG provides products and services for internal logistics worldwide. Its products include rollers, drives, conveyors and sorters, and pallet handling products. The company offers driven conveyor rollers, control systems, and drum motors; conveyors and sorters, including cross-belt sorters, belt curves, modular conveyor platforms, and modular pallet conveyor platforms; and flow storage solutions for pallets and parcel packaging. Its products and services are used in courier, express, and parcel; airport; e-commerce; tire and automotive; fashion; food and beverage; and distribution and warehouse industries, as well as in various other manufacturing industries. The company serves plant manufacturers, original equipment manufacturers, system integrators, and end users. Interroll Holding AG was founded in 1959 and is headquartered in Sant'Antonino, Switzerland.


Interroll Holding AG is a Swiss manufacturer of products and services for in-house logistics. The Compa

In [ ]:
text, sentence_tag = tag_sentences(text_manual, text_yahoo, text_bloomberg, text_marketscreener)

---------------------
First sentences
Interroll Holding AG provides products and services for internal logistics worldwide.
Interroll Holding AG is a Swiss manufacturer of products and services for in-house logistics.
---------------------
Sales
---------------------
Services
---------------------
Its products include rollers, drives, conveyors and sorters, and pallet handling products.
---------------------
The company offers driven conveyor rollers, control systems, and drum motors; conveyors and sorters, including cross-belt sorters, belt curves, modular conveyor platforms, and modular pallet conveyor platforms; and flow storage solutions for pallets and parcel packaging.
---------------------
Its products and services are used in courier, express, and parcel; airport; e-commerce; tire and automotive; fashion; food and beverage; and distribution and warehouse industries, as well as in various other manufacturing industries.
---------------------
The company serves plant manufacturer

In [ ]:
text

"Its products include rollers, drives, conveyors and sorters, and pallet handling products. The company offers driven conveyor rollers, control systems, and drum motors; conveyors and sorters, including cross-belt sorters, belt curves, modular conveyor platforms, and modular pallet conveyor platforms; and flow storage solutions for pallets and parcel packaging. Its products and services are used in courier, express, and parcel; airport; e-commerce; tire and automotive; fashion; food and beverage; and distribution and warehouse industries, as well as in various other manufacturing industries. The company serves plant manufacturers, original equipment manufacturers, system integrators, and end users. Interroll Holding AG was founded in 1959 and is headquartered in Sant'Antonino, Switzerland. The Company's business is structured into four product groups. The Rollers group focuses on the production of conveyor rollers. The Drives group manufactures motors and drivers for conveyors. The Con

In [ ]:
text_manual = ''
text_yahoo = ""
text_bloomberg = ''
text_marketscreener = '''Dürr specializes in the design and construction of turnkey painting shops for the automotive sector. The group also manufactures and markets machines for woodworking. Net sales break down by activity as follows:

- design and construction of painting shops (48.3%). Dürr also produces assembly, paint spray and air purification systems;

- production of woodworking machinery (32.9%);

- manufacturing of measurement systems (18,8%): for the vehicles assembly. The group also manufactures and sells filtration and industrial cleaning systems for the automotive, aeronautic, electricity, electronic and mechanic sectors.

Net sales are distributed geographically as follows: Germany (13.1%), Europe (29.3%), Asia/Africa/Australia (30.5%) and Americas (27.1%).Number of employees : 16 493 people.'''
text, sentence_tag = tag_sentences(text_manual, text_yahoo, text_bloomberg, text_marketscreener)
  
first_sentences = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in 'first sentence')
sentences = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in ['segments.services','strictly services'])
sales = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in ['sales','money.sales'])
relations = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in 'client.shareholders.subsidiary')
other_numbers = " ".join(sentence for sentence,tag in sentence_tag.items() if tag in 'other.numbers')

#first sentence summary - minimum length: either 15% or 25
min_len = round(0.15 * len(first_sentences))
print("15% is " + str(min_len))
if min_len > 25:
  min_len = 25
print(min_len)
if min_len > 10:
  fs_inputs = tokenizer([first_sentences], return_tensors='pt')
  fs_summary_ids = model.generate(fs_inputs['input_ids'], num_beams=4, no_repeat_ngram_size=2, min_length=min_len, early_stopping=True) 
  fs_summary = tokenizer.decode(fs_summary_ids[0], skip_special_tokens=True)
  pprint.pprint(fs_summary)
  doc_fs = create_NLP_object(fs_summary)
  fs_summary_sentences = [token.text for token in doc_fs.sents]
  for s_idx, sentence in enumerate(fs_summary_sentences):
    if re.search('For more information|Back to|back to |For confidential.? support|founded in \d{4}', sentence):
      print(sentence)
      fs_summary = fs_summary.replace(sentence,'')
      fs_summary_sentences.pop(s_idx)

  if re.search('The Company is based in|The company is based in|It is based in', fs_summary_sentences[-1]):
    fs_summary = fs_summary.replace(fs_summary_sentences[-1],'')

  if fs_summary[-1]!='.' and fs_summary[-1]!=' ':
    fs_summary = fs_summary + "."
else:
  fs_summary = ''

pprint.pprint(fs_summary)

#segments & services length summary - minimum length: either 15% or 100
summary_len = ''
total_summary = ''
if sentences:
  inputs = tokenizer([sentences], return_tensors='pt', max_length= len(sentences), truncation=True)
  min_len = round(0.15 * len(sentences))
  print("\n15% is " + str(min_len))
  if min_len > 100:
    min_len = 100
    
  summary_ids_len = model.generate(inputs['input_ids'], num_beams=4, no_repeat_ngram_size=2, min_length = min_len, max_length=600, early_stopping=True) 
  summary_len = tokenizer.decode(summary_ids_len[0], skip_special_tokens=True)
  print('Services summary:')
  pprint.pprint(summary_len)
  if summary_len[-1]!='.' and summary_len[-1]!= ' ':
    summary_len = summary_len + "."

  doc_summary = create_NLP_object(summary_len)
  summary_sentences = [token.text for token in doc_summary.sents]
  for s_idx, sentence in enumerate(summary_sentences):
    if re.search('For more information|for more|Back to|back to |For confidential.? support|WARNING:|Warning:', sentence):
      print(sentence)
      summary_len = summary_len.replace(sentence,'')
      summary_sentences.pop(s_idx)
    
    if re.search(r'In \d{4}, (.*)\d|recorded a net profit|recorded a net loss|total assets stand|reported a net profit|reported a net loss|share price', sentence):
      print(sentence)
      summary_len = summary_len.replace(sentence,'')
      summary_sentences.pop(s_idx)
  
  if re.search('The Company is based in|The company is based in|It is based in', summary_sentences[-1]):
    summary_len = summary_len.replace(summary_sentences[-1],'')

  total_summary = fs_summary + " " + summary_len
  total_summary = total_summary.replace("\xa0"," ")
  total_summary = remove_spaces(total_summary)
  print('\n')
  pprint.pprint(total_summary)
else:
  print('\nNo services found')
  text = first_sentences + " " + text
  text = remove_spaces(text)
  inputs = tokenizer([text], return_tensors='pt', max_length= len(text), truncation=True)
  min_len = round(0.20 * len(text))
  print("\n20% is " + str(min_len))
  if min_len > 100:
    min_len = 100
  summary_ids_len = model.generate(inputs['input_ids'], num_beams=4, no_repeat_ngram_size=2, min_length = min_len, max_length=300, early_stopping=True) 
  total_summary = tokenizer.decode(summary_ids_len[0], skip_special_tokens=True)
  print('Total summary:')
  pprint.pprint(total_summary)
  if total_summary[-1]!='.':
    total_summary = total_summary + "."
  doc_summary = create_NLP_object(total_summary)
  summary_sentences = [token.text for token in doc_summary.sents]
  for s_idx, sentence in enumerate(summary_sentences):
    if re.search('For more information|for more|Back to|back to |For confidential.? support|WARNING:|Warning:|is based in', sentence):
      print(sentence)
      total_summary = total_summary.replace(sentence,'')
      summary_sentences.pop(s_idx)

    if re.search(r'In \d{4}, (.*)\d|recorded a net profit|recorded a net loss|total assets stand|reported a net profit|reported a net loss|share price', sentence):
      print(sentence)
      total_summary = total_summary.replace(sentence,'')  
      summary_sentences.pop(s_idx)

  if re.search('The Company is based in|The company is based in|It is based in', summary_sentences[-1]):
    total_summary = total_summary.replace(summary_sentences[-1],'')  
  
print('\ntotal summary:')
print(total_summary)
  

---------------------
First sentences
Dürr specializes in the design and construction of turnkey painting shops for the automotive sector.
---------------------
Sales
Net sales break down by activity as follows: - design and construction of painting shops (48.3%). Dürr also produces assembly, paint spray and air purification systems; - production of woodworking machinery (32.9%); - manufacturing of measurement systems (18,8%): for the vehicles assembly. The group also manufactures and sells filtration and industrial cleaning systems for the automotive, aeronautic, electricity, electronic and mechanic sectors. Net sales are distributed geographically as follows: Germany (13.1%), Europe (29.3%), Asia/Africa/Australia (30.5%) and Americas (27.1%).
---------------------
Services
---------------------
---------------------
Relations
[]
---------------------
Monetary & Sales
---------------------
Other numbers
15% is 15
15
('Dürr specializes in the design and construction of turnkey painting